<a href="https://colab.research.google.com/github/mariell-morven/Hipotesis_Spotify/blob/main/Pruebas%20de%20Significancia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**¿Por qué se escogió el test de Wilcoxon (Mann-Whitney U test)?**
1. *Tipo de datos*: El test es una prueba no paramétrica, es decir, se utiliza cuando los datos no se distribuyen normalmente.
2. *Robustez*: Es menos sensible a las desviaciones de la normalidad y más robusto ante valores atípicos.
3. *Tamaño de la muestra*: Adecuado para muestras pequeñas.

# **Test de Wilcoxon (Mann-Whitney U test)**
---


*Prueba de significancia entre el promedio de streams de la categoría "alto" y "bajo" de cada característica musical, para rechazar o no su hipótesis nula, y por tanto aceptar o no su hipótesis alternativa, respectivamente:*
<br></br>
**h0**
> No existe diferencia significativa entre el promedio de streams de la categoría "alto" de [característica] y el promedio de los streams de la categoría "bajo" de [característica]

**h1**
> Existe una diferencia significativa entre el promedio de streams de la categoría "alto" de [característica] y el promedio de los streams de la categoría "bajo" de [característica]


**Características:**
* Danceability
* Valence
* Energy
* Acousticness
* Instrumentalness
* Liveness
* Speechiness



In [ ]:
from google.cloud import bigquery
from scipy.stats import mannwhitneyu
import numpy as np

# Autenticación
from google.colab import auth
auth.authenticate_user()

# Configuración del cliente de BigQuery
project_id = 'proyecto-no2-hipotesis'
client = bigquery.Client(project=project_id)

# Diccionario de nombres de características
nombres_caracteristicas = {
    'category_dance': 'Danceability',
    'category_valence': 'Valence',
    'category_energy': 'Energy',
    'category_acoustic': 'Acousticness',
    'category_instrumental': 'Instrumentalness',
    'category_live': 'Liveness',
    'category_speech': 'Speechiness'
}

# Lista de características a analizar
características = list(nombres_caracteristicas.keys())

for característica in características:
    # Consulta SQL para obtener los datos de streams por característica
    query = f"""
    SELECT {característica}, streams_clean
    FROM tracks.master
    """
    # Ejecutar la consulta
    query_job = client.query(query)

    # Almacenar streams en listas por categoría
    streams_alto = []
    streams_bajo = []

    # Filtrar y almacenar datos numéricos en listas
    for row in query_job.result():
        category = row[característica]
        streams = row['streams_clean']
        if category == 'alto':
          streams_alto.append(streams)
        elif category == 'bajo':
          streams_bajo.append(streams)

    # Calcular promedios por categoría y redondear a 3 decimales
    avg_streams_alto = round(np.mean(streams_alto), 3)
    avg_streams_bajo = round(np.mean(streams_bajo), 3)

    # Aplicar el test de Wilcoxon (Mann-Whitney U test) a cada característica
    statistic, p_value = mannwhitneyu(streams_alto, streams_bajo, alternative='two-sided')

    # Definir nivel de significancia
    alpha = 0.05

    # Obtener el nombre actual y el nuevo nombre de la característica
    nombre_actual = característica
    nombre_nuevo = nombres_caracteristicas[característica]

    # Mostrar los resultados del test para cada característica
    print(f"\nAnálisis para la característica: \033[1m{nombre_nuevo}\033[0m")
    if p_value < alpha:
        print("Se rechaza la hipótesis nula: Existe una diferencia significativa entre las categorías ''alto'' y ''bajo'")
    else:
        print("No se puede rechazar la hipótesis nula: No hay una diferencia significativa entre las categorías ''alto'' y ''bajo'")

    # Mostrar los valores del test y promedios obtenidos para cada característica
    print(f"Estadístico del test: {statistic}")
    print(f"P-valor: {p_value}")
    print(f"Promedio de streams 'alto' de : {avg_streams_alto}")
    print(f"Promedio de streams 'bajo' de : {avg_streams_bajo}")



Análisis para la característica: Danceability
Se rechaza la hipótesis nula: Existe una diferencia significativa entre las categorías ''alto'' y ''bajo'
Estadístico del test: 76083.0
P-valor: 0.02160022464341392
Promedio de streams 'alto' de : 422353932.591
Promedio de streams 'bajo' de : 544616352.893

Análisis para la característica: Valence
No se puede rechazar la hipótesis nula: No hay una diferencia significativa entre las categorías ''alto'' y ''bajo'
Estadístico del test: 80538.5
P-valor: 0.28023114674059835
Promedio de streams 'alto' de : 470882779.017
Promedio de streams 'bajo' de : 528485446.017

Análisis para la característica: Energy
No se puede rechazar la hipótesis nula: No hay una diferencia significativa entre las categorías ''alto'' y ''bajo'
Estadístico del test: 81660.0
P-valor: 0.43932393592841257
Promedio de streams 'alto' de : 498501279.916
Promedio de streams 'bajo' de : 519305102.801

Análisis para la característica: Acousticness
No se puede rechazar la hipótesi

En resumen, los resultados de las hipótesis nulas y alternativas rechazadas o no, es el siguiente:

Hipótesis          | h0 | h1 |
-------------------|-----|-----
Danceability     | ❌ | ✔
Valence     | ✔ | ❌
Energy    | ✔ | ❌
Acousticness     | ✔ | ❌
Instrumentalness  | ✔ | ❌
Liveness      | ✔ | ❌
Speechiness     | ❌ | ✔

# Tabla resumen

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import google.colab.data_table as dt

# Autenticación con Google
auth.authenticate_user()

# Crear cliente de BigQuery
project_id = 'proyecto-no2-hipotesis'
client = bigquery.Client(project=project_id)

# Query SQL para seleccionar las columnas necesarias
query = """
SELECT streams_clean, bpm, danceability__, valence__, energy__, acousticness__, instrumentalness__, liveness__, speechiness__
FROM proyecto-no2-hipotesis.tracks.master
"""

# Ejecutar la consulta en BigQuery y obtener los resultados
query_job = client.query(query)
df = query_job.to_dataframe()

# Calcular valores estadísticos
summary_df = df.agg(['min', 'max', 'mean', 'sum', 'std'])

# Redondear los resultados a 3 decimales
summary_df = summary_df.round(3)

# Renombrar columnas en el DataFrame summary_df
new_columns = {
    'streams_clean': 'Streams',
    'bpm': 'BPM',
    'danceability__': 'Danceability',
    'valence__': 'Valence',
    'energy__': 'Energy',
    'acousticness__': 'Acousticness',
    'instrumentalness__': 'Instrumentalness',
    'liveness__': 'Liveness',
    'speechiness__': 'Speechiness'
}
summary_df = summary_df.rename(columns=new_columns)

# Renombrar filas en el DataFrame summary_df
new_index = {
    'min': 'Valor Mínimo',
    'max': 'Valor Máximo',
    'mean': 'Promedio',
    'sum': 'Suma',
    'std': 'Desviación Estándar',
}
summary_df = summary_df.rename(index=new_index)

# Mostrar el DataFrame como una tabla interactiva
dt.DataTable(summary_df)


,Streams,BPM,Danceability,Valence,Energy,Acousticness,Instrumentalness,Liveness,Speechiness
Valor Mínimo,2.762000e+03,65.000,23.000,4.000,9.000,0.000,0.000,3.000,2.000
Valor Máximo,3.703895e+09,206.000,96.000,97.000,97.000,97.000,91.000,97.000,64.000
Promedio,5.141151e+08,122.571,66.963,51.365,64.248,27.118,1.586,18.198,10.142
Suma,4.884093e+11,116442.000,63615.000,48797.000,61036.000,25762.000,1507.000,17288.000,9635.000
Desviación Estándar,5.673801e+08,28.096,14.636,23.488,16.566,26.015,8.423,13.723,9.925
